In [1]:
import zipfile
import os

# ZIP dosyasının yolu
zip_path = '/content/archive.zip'

# ZIP'i çıkarma klasörü
extract_path = '/content/brain_tumor_dataset'

# ZIP dosyasını çıkar
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("✅ Dosyalar çıkarıldı:", os.listdir(extract_path))

✅ Dosyalar çıkarıldı: ['brain_tumor_dataset', 'yes', 'no']


In [2]:
import os
import shutil
import random

def split_data(source_dir, target_dir, split_ratio=(0.7, 0.15, 0.15)):
    categories = ['yes', 'no']
    for category in categories:
        files = os.listdir(os.path.join(source_dir, category))
        random.shuffle(files)

        total = len(files)
        train_count = int(split_ratio[0] * total)
        val_count = int(split_ratio[1] * total)

        splits = {
            'train': files[:train_count],
            'val': files[train_count:train_count+val_count],
            'test': files[train_count+val_count:]
        }

        for split_name, split_files in splits.items():
            split_path = os.path.join(target_dir, split_name, category)
            os.makedirs(split_path, exist_ok=True)
            for file in split_files:
                src = os.path.join(source_dir, category, file)
                dst = os.path.join(split_path, file)
                shutil.copy(src, dst)

# Kaynak klasör (çıkarılan zip’in içindeki yes/no)
source_dir = '/content/brain_tumor_dataset'
# Yeni veri yapısının oluşturulacağı yer
target_dir = '/content/dataset'

# Bölme işlemi
split_data(source_dir, target_dir)

# Kontrol
for split in ['train', 'val', 'test']:
    for cls in ['yes', 'no']:
        path = os.path.join(target_dir, split, cls)
        print(f"{split}/{cls}: {len(os.listdir(path))} görüntü")


train/yes: 139 görüntü
train/no: 89 görüntü
val/yes: 40 görüntü
val/no: 27 görüntü
test/yes: 45 görüntü
test/no: 29 görüntü


In [3]:
import torch
from torchvision import transforms, datasets
from torch.utils.data import DataLoader

image_size = 224
batch_size = 32

transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # 3 kanal RGB
])

# Datasetleri oluştur
data_dir = "/content/dataset"

train_dataset = datasets.ImageFolder(os.path.join(data_dir, 'train'), transform=transform)
val_dataset = datasets.ImageFolder(os.path.join(data_dir, 'val'), transform=transform)
test_dataset = datasets.ImageFolder(os.path.join(data_dir, 'test'), transform=transform)

# DataLoader'lar
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Kontrol: sınıf isimleri
print("Sınıflar:", train_dataset.classes)
print("Toplam eğitim görüntüsü:", len(train_dataset))

Sınıflar: ['no', 'yes']
Toplam eğitim görüntüsü: 228


In [4]:
!pip install timm

In [5]:
import timm
import torch.nn as nn

# Cihaz kontrolü
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Kullanılan cihaz:", device)

# ViT modeli (timm üzerinden)
model = timm.create_model('vit_base_patch16_224', pretrained=True)
model.head = nn.Linear(model.head.in_features, 2)  # 2 sınıf: tümör var / yok
model.to(device)

Kullanılan cihaz: cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (patch_drop): Identity()
  (norm_pre): Identity()
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (q_norm): Identity()
        (k_norm): Identity()
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): Identity()
      (drop_path1): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (norm): Identity(

In [6]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Eğitim fonksiyonu
def train(model, loader, optimizer, criterion):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    acc = 100 * correct / total
    return running_loss / len(loader), acc

# Doğrulama fonksiyonu
def validate(model, loader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    acc = 100 * correct / total
    return running_loss / len(loader), acc

In [7]:
num_epochs = 10

for epoch in range(num_epochs):
    train_loss, train_acc = train(model, train_loader, optimizer, criterion)
    val_loss, val_acc = validate(model, val_loader, criterion)

    print(f"Epoch {epoch+1}/{num_epochs} | "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}% | "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")

Epoch 1/10 | Train Loss: 1.0679, Train Acc: 62.72% | Val Loss: 0.6772, Val Acc: 59.70%
Epoch 2/10 | Train Loss: 0.7208, Train Acc: 53.51% | Val Loss: 0.6788, Val Acc: 59.70%
Epoch 3/10 | Train Loss: 0.6976, Train Acc: 58.77% | Val Loss: 0.6235, Val Acc: 76.12%
Epoch 4/10 | Train Loss: 0.5618, Train Acc: 71.93% | Val Loss: 0.6307, Val Acc: 79.10%
Epoch 5/10 | Train Loss: 0.5238, Train Acc: 76.32% | Val Loss: 0.5467, Val Acc: 88.06%
Epoch 6/10 | Train Loss: 0.4544, Train Acc: 77.19% | Val Loss: 0.6364, Val Acc: 85.07%
Epoch 7/10 | Train Loss: 0.3998, Train Acc: 83.77% | Val Loss: 0.3246, Val Acc: 89.55%
Epoch 8/10 | Train Loss: 0.2684, Train Acc: 86.40% | Val Loss: 0.2099, Val Acc: 94.03%
Epoch 9/10 | Train Loss: 0.2004, Train Acc: 92.54% | Val Loss: 0.2324, Val Acc: 86.57%
Epoch 10/10 | Train Loss: 0.1857, Train Acc: 90.79% | Val Loss: 1.1269, Val Acc: 58.21%


In [8]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

def evaluate_on_test(model, loader):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)

            if outputs.ndim == 4:
              outputs = outputs.mean(dim=[2, 3])
            _, predicted = torch.max(outputs, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    print("📋 Sınıflandırma Raporu:")
    print(classification_report(all_labels, all_preds, target_names=train_dataset.classes))

    print("🧩 Confusion Matrix:")
    print(confusion_matrix(all_labels, all_preds))

evaluate_on_test(model, test_loader)

📋 Sınıflandırma Raporu:
              precision    recall  f1-score   support

          no       0.59      1.00      0.74        29
         yes       1.00      0.56      0.71        45

    accuracy                           0.73        74
   macro avg       0.80      0.78      0.73        74
weighted avg       0.84      0.73      0.73        74

🧩 Confusion Matrix:
[[29  0]
 [20 25]]


In [9]:
model2 = timm.create_model('deit_base_patch16_224', pretrained=True)
model2.head = nn.Linear(model2.head.in_features, 2)
model2.to(device)

# Yeni optimizer
optimizer2 = optim.Adam(model2.parameters(), lr=0.0001)

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

In [10]:
for epoch in range(num_epochs):
    train_loss, train_acc = train(model2, train_loader, optimizer2, criterion)
    val_loss, val_acc = validate(model2, val_loader, criterion)

    print(f"[DeiT] Epoch {epoch+1}/{num_epochs} | "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}% | "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")

[DeiT] Epoch 1/10 | Train Loss: 0.6182, Train Acc: 67.54% | Val Loss: 0.3509, Val Acc: 83.58%
[DeiT] Epoch 2/10 | Train Loss: 0.2276, Train Acc: 92.11% | Val Loss: 0.4439, Val Acc: 73.13%
[DeiT] Epoch 3/10 | Train Loss: 0.1534, Train Acc: 91.67% | Val Loss: 0.0741, Val Acc: 95.52%
[DeiT] Epoch 4/10 | Train Loss: 0.0415, Train Acc: 98.25% | Val Loss: 0.0823, Val Acc: 98.51%
[DeiT] Epoch 5/10 | Train Loss: 0.0080, Train Acc: 100.00% | Val Loss: 0.1246, Val Acc: 97.01%
[DeiT] Epoch 6/10 | Train Loss: 0.0065, Train Acc: 100.00% | Val Loss: 0.1260, Val Acc: 97.01%
[DeiT] Epoch 7/10 | Train Loss: 0.0045, Train Acc: 99.56% | Val Loss: 0.1669, Val Acc: 94.03%
[DeiT] Epoch 8/10 | Train Loss: 0.0005, Train Acc: 100.00% | Val Loss: 0.1624, Val Acc: 95.52%
[DeiT] Epoch 9/10 | Train Loss: 0.0012, Train Acc: 100.00% | Val Loss: 0.1774, Val Acc: 95.52%
[DeiT] Epoch 10/10 | Train Loss: 0.0002, Train Acc: 100.00% | Val Loss: 0.1879, Val Acc: 95.52%


In [11]:
evaluate_on_test(model2, test_loader)

📋 Sınıflandırma Raporu:
              precision    recall  f1-score   support

          no       0.97      0.97      0.97        29
         yes       0.98      0.98      0.98        45

    accuracy                           0.97        74
   macro avg       0.97      0.97      0.97        74
weighted avg       0.97      0.97      0.97        74

🧩 Confusion Matrix:
[[28  1]
 [ 1 44]]


In [12]:
# from numba import cuda
# device = cuda.get_current_device()
# device.reset()

In [13]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# print("Kullanılan cihaz:", device)

In [14]:
model3 = timm.create_model('cait_s24_224', pretrained=True)
model3.head = nn.Linear(model3.head.in_features, 2)
model3.to(device)

optimizer3 = optim.Adam(model3.parameters(), lr=0.0001)

model.safetensors:   0%|          | 0.00/188M [00:00<?, ?B/s]

In [15]:
for epoch in range(num_epochs):
    train_loss, train_acc = train(model3, train_loader, optimizer3, criterion)
    val_loss, val_acc = validate(model3, val_loader, criterion)

    print(f"[Swin] Epoch {epoch+1}/{num_epochs} | "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}% | "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")

[Swin] Epoch 1/10 | Train Loss: 0.8260, Train Acc: 61.40% | Val Loss: 1.0444, Val Acc: 49.25%
[Swin] Epoch 2/10 | Train Loss: 0.4344, Train Acc: 76.32% | Val Loss: 0.1894, Val Acc: 91.04%
[Swin] Epoch 3/10 | Train Loss: 0.1736, Train Acc: 92.98% | Val Loss: 0.0187, Val Acc: 100.00%
[Swin] Epoch 4/10 | Train Loss: 0.0790, Train Acc: 97.81% | Val Loss: 0.0612, Val Acc: 98.51%
[Swin] Epoch 5/10 | Train Loss: 0.0269, Train Acc: 98.68% | Val Loss: 0.0040, Val Acc: 100.00%
[Swin] Epoch 6/10 | Train Loss: 0.0079, Train Acc: 100.00% | Val Loss: 0.0356, Val Acc: 98.51%
[Swin] Epoch 7/10 | Train Loss: 0.0349, Train Acc: 98.68% | Val Loss: 0.2547, Val Acc: 88.06%
[Swin] Epoch 8/10 | Train Loss: 0.1515, Train Acc: 93.86% | Val Loss: 0.0265, Val Acc: 98.51%
[Swin] Epoch 9/10 | Train Loss: 0.0592, Train Acc: 98.68% | Val Loss: 0.0351, Val Acc: 98.51%
[Swin] Epoch 10/10 | Train Loss: 0.0052, Train Acc: 100.00% | Val Loss: 0.1430, Val Acc: 95.52%


In [17]:
evaluate_on_test(model3, test_loader)

📋 Sınıflandırma Raporu:
              precision    recall  f1-score   support

          no       0.97      1.00      0.98        29
         yes       1.00      0.98      0.99        45

    accuracy                           0.99        74
   macro avg       0.98      0.99      0.99        74
weighted avg       0.99      0.99      0.99        74

🧩 Confusion Matrix:
[[29  0]
 [ 1 44]]
